In [1]:
import hyperspy.api as hs
import plotly.graph_objects as go
import numpy as np

from utils.get_raw_data import locate_raw_data, get_multiple_data_arrays
from helper_files.plotting import plotly_plot
from helper_files.gaussian_fitting import gaussian, n_gaussians, fit_n_peaks_to_gaussian, area_under_peak
from helper_files.calibration import calibrate_channel_width_two_peaks, channel_to_keV
from helper_files.spectrum_dict import init_known_spectrum

emsa = locate_raw_data()

In [2]:
s = init_known_spectrum(
    name="GaAs at 30kV",
    filepath=emsa[7],
    start_str="#SPECTRUM    : Spectral Data Starts Here",
    stop_str="#ENDOFDATA   : ",
    line_endings="\n",
    delimiter=", ",
    # calibrate on Ga_La and As_Ka
    peaks_keV=[1.098, 10.5436],
    peaks_names=['Ga L&#945;</sub>', 'As K&#945;</sub>'],
    peaks_channel=[131, 1072, 944, 1044],
)

Reading C:\Users\Brynjar\Documents\Masteroppgave\2022-09-06_EDS-Apreo\exports\GaAs_30kV.emsa
The first line looks like this: '#FORMAT      : EMSA/MAS Spectral Data File\n'
Reading from line 42 to 2091.
2048 data points, first entry = [-0.2, 0.0], last entry = [20.27, 52.0]



In [3]:
ga30 = get_multiple_data_arrays(filters=["GaAs_30"])[0][1]
# %matplotlib qt
# s = hs.load(emsa[6], signal_type="EDS_SEM")
# s.add_elements(["Ga", "As"])
# s.plot(True, navigator='slider')


In [4]:
# Ga La, Ga Lb1, As La, As Lb1, Ga Ka, Ga Kb, As Ka, As Kb
peaks = [['Ga', 'La'], ['Ga', 'Lb1'], ['As', 'La'], ['As', 'Lb1'], ['Ga', 'Ka'], ['Ga', 'Kb'], ['As', 'Ka'], ['As', 'Kb']]
peaks_names = ['Ga L&#945;', 'Ga L&#946;<sub>1</sub>', 'As L&#945;', 'As L&#946;<sub>1</sub>', 'Ga K&#945;', 'Ga K&#946;', 'As K&#945;', 'As K&#946;'],
ga_ = hs.material.elements['Ga'].Atomic_properties.Xray_lines.as_dictionary()
as_ = hs.material.elements['As'].Atomic_properties.Xray_lines.as_dictionary()

In [5]:
peaks_keV = []
peaks_weight = []
print(f"{'line':<8} {'energy':<8} {'weight':<6}")

for p in peaks:
    if p[0] == 'Ga':
        print(f"{p[0]:<3} {p[1]:<4} {ga_[p[1]]['energy (keV)']:<8} {ga_[p[1]]['weight']:<6}")
        peaks_keV.append(ga_[p[1]]['energy (keV)'])
        peaks_weight.append(ga_[p[1]]['weight'])
    else:
        print(f'{p[0]:<3} {p[1]:<4} {as_[p[1]]["energy (keV)"]:<8} {as_[p[1]]["weight"]:<6}')
        peaks_keV.append(as_[p[1]]['energy (keV)'])
        peaks_weight.append(as_[p[1]]['weight'])
peaks_keV

line     energy   weight
Ga  La   1.098    1.0   
Ga  Lb1  1.1249   0.16704
As  La   1.2819   1.0   
As  Lb1  1.3174   0.16704
Ga  Ka   9.2517   1.0   
Ga  Kb   10.2642  0.1287
As  Ka   10.5436  1.0   
As  Kb   11.7262  0.14589


[1.098, 1.1249, 1.2819, 1.3174, 9.2517, 10.2642, 10.5436, 11.7262]

In [6]:
fig_initial = go.Figure()
channels = list(range(0, len(ga30)))  # 2048
fig_initial.add_trace(go.Scatter(x=channels, y=ga30, name="Raw Data"))



In [7]:
# fitting

fit_vals = fit_n_peaks_to_gaussian(
    x=s["channel"],
    y=s["intensity"],
    guessed_peaks=s["peaks_channel"],
)
s["fit_params"] = fit_vals[0]
s["fit_cov"] = fit_vals[1]
# update the channel values of the peaks
s["peaks_channel"] = s["fit_params"][1::3]
# addin the fitted gaussian to the spectrum-dictionary
s["intensity_fit"] = n_gaussians(s["channel"], *fit_vals[0])

print(f'Fitted peaks at channel: {s["peaks_channel"]}')


Fitted peaks at channel: [ 130.56740439 1072.45815316  943.74174496 1045.33579055]


In [8]:
fig_fit = plotly_plot(
    s["channel"],
    s["intensity"],
    y_fit=s["intensity_fit"],
    vlines=s["peaks_channel"],
    vlines_name=s["peaks_names"],
    title=f"Gaussian fitting of {s['name']}",
)
# fig_fit = plotly_plot(x=s['channel'], fig=fig_fit, fit_params=s['fit_params'], title=f"Gaussian fitting of {s['name']}")
# fig_fit.show()

calib = calibrate_channel_width_two_peaks(s["peaks_channel"], s["peaks_keV"])
s["dispersion"] = calib[0]
s["offset"] = calib[1]
s["kev_calibrated"] = (s["channel"] - s["offset"]) * s["dispersion"]

The calibration factor is: 0.0100283 keV/channel, with 21.078 channels zero offset


In [9]:
print("Peak name | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak")
for i in range(len(s["fit_params"]) // 3):
    try:
        peak_name = s["peaks_names"][i]
    except (
        IndexError,
        TypeError,
    ):  # if s['peaks_names'] is None or not as long as the number of peaks
        peak_name = f"peak {i}"
    mu = channel_to_keV(s, value=s['fit_params'][i*3+1])
    amp = s['fit_params']
    fwhm = channel_to_keV(s, value=s['fit_params'][i * 3 + 2] * 2 * (np.log(2) * 2)**0.5, use_offset=False)
    area = area_under_peak(s['fit_params'][i*3+1], s['fit_params'][2 + 3 * i], s['fit_params'][3 * i])
    print(f"{peak_name:<9} | {mu:<24.4f} | {amp[i*3]:<18.4f} | {fwhm:<32.4f} | {area:.6f}")


s["peaks_keV_fitted"] = channel_to_keV(s, value=s['peaks_channel'])



Peak name | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak
Ga L&#945;</sub> | 1.0980                   | 0.9979             | 0.0750                           | 0.995223
As K&#945;</sub> | 10.5436                  | 0.2013             | 0.1724                           | 0.200806
peak 2    | 9.2528                   | 0.3561             | 0.1633                           | 0.355143
peak 3    | 10.2716                  | 0.0534             | 0.2027                           | 0.053281


In [15]:
# for plotting Ga Ka with theoretical Ga Ka1 and Ga Ka2
# lines = [HyperSpy, Ga Ka1, Ga Ka2, calibrated Ga Ka]
vertical_lines = [9.22482, 9.25174, 9.2517,  9.2528]
vertical_lines_names = ['Ga K&#945;<sub>2</sub>', 'Ga K&#945;<sub>1</sub>', 'Ga K&#945;<sub>HyperSpy</sub>', 'Ga K&#945;<sub>Calibrated center</sub>']

fig_calib = plotly_plot(
x=s["kev_calibrated"],
y_named=[s["intensity"], "intensity"],
vlines=vertical_lines,
vlines_name=vertical_lines_names,
title=s['name'],
xaxis_title="Energy [keV]",
start=925, #  index
stop=964, #  index
)

fig_calib.update_layout(yaxis_range=[-0.025,0.4])
fig_calib.update_layout(showlegend=False)
fig_calib.show()
# fig_calib.write_html("../plots/GaAs30kV-Ga-K-lines.html")
# fig_calib.write_image("../plots/GaAs30kV-Ga-K-lines.png")